<a href="https://colab.research.google.com/github/BearsOnMars/baby-alpha/blob/main/Scraping_Financial_Data_of_The_Coca_Cola_Company_(KO)_from_Yahoo_Finance.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Scraping data using 'hidden APIs' method and 'extracting embedded json formatted data' method. These can be easily converted to a python dictionary. Standard libraries are used

In [ ]:
import re
import json
import csv
from io import StringIO
# Installing Beautiful Soup and Requests package. Note the exclamation mark at the beginning
# In Google Colab, we can run a console command by starting with a ! followed by the command
!pip install bs4
!pip install requests
from bs4 import BeautifulSoup
import requests

Going to Yahoo Finance now, entering relevant stock ticker in search. Data can be extracted for any tab i.e. statistics, historical data, financials, options. went and searched for The Coca-Cola company. copying the url for the tabs we are interested in scraping. In place of the stock ticker symbol such as KO for Coca-Cola in these similar looking urls, if we use {}, it becomes easier at later stages to insert any new symbol with string formatting at a later stage.

In [3]:
url_stats = 'https://finance.yahoo.com/quote/{}/key-statistics?p={}'
url_profile = 'https://finance.yahoo.com/quote/{}/profile?p={}'
url_financials = 'https://finance.yahoo.com/quote/{}/financials?p={}'

In [4]:
# Assigning KO to stock Coca Cola so as to look it up on Yahoo finance for the ticker
stock = 'KO'

Scraping Financial Statements. Using git request(?) even data which can not be seen in the website can be downloaded. Using financials url template to request the stock data using request library. Response is assigned to a response variable

In [5]:
response = requests.get(url_financials.format(stock, stock))

In [6]:
# HTML data is parsed using BeautifulSoup and default HTML parser
soup = BeautifulSoup(response.text, 'html.parser')

Going back to the webpage now to look at raw data by right-click 'view page source'. The webpage is dynamically loading data from the javascript function contained within the script tag. The comment /* -- Data --*/ contains all data which is dynamically being loaded on to the page. Since, the data is embedded in a javascript function, we need to get the script tag first, then extract the contents from it, chop the edges off so that we are just left with the json formatted string, which can then be converted to a python dictionary. There is no unique id tag for the script, so we need to use a text pattern with regular expressions

In [7]:
pattern = re.compile(r'\s--\sData\s--\s')

using this pattern now find the script element that has text which matches this pattern. return the contents of this element as a list. grab the first and only item in the list


In [8]:
script_data = soup.find('script', text=pattern).contents[0]

In [ ]:
# Printing the beginning and the ending 500 characters of the script data
# Notice the colon at start in beginning 
script_data[:500]

In [ ]:
# Notice the negative sign and the colon at the end while printing ending
script_data[-500:]

The problem now is that we have this javascript code wrapping the data which we actually want. To do that we need to find the boundaries of this slice. We can see in the beginning 500 char, it starts with "context" the starting position would be 2 characters behind the beginning of the word 'context'

In [11]:
start = script_data.find("context")-2

the ending position would be 12 characters from the end. After that patch this sliced text into the json loads function, to get a python dictionary back

In [12]:
json_data = json.loads(script_data[start:-12])

In [ ]:
# Using the keys method, it could be found where data for financial statements is loacted in the context, dispacher, stores and QuoteSummaryStore
json_data['context'].keys()

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

In [15]:
annual_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistory']['incomeStatementHistory']
quarterly_is = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['incomeStatementHistoryQuarterly']['incomeStatementHistory']

annual_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistory']['cashflowStatements']
quarterly_cf = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['cashflowStatementHistoryQuarterly']['cashflowStatements']

annual_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistory']['balanceSheetStatements']
quarterly_bs = json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['balanceSheetHistoryQuarterly']['balanceSheetStatements']

Exploring income statement variable to discern the pattern and basing the rest of the code on it

In [ ]:
print(annual_is[0])

Each statement contains a list of dictionary that contain account name and variety of number formats

In [ ]:
annual_is[0]['operatingIncome']

Using 'for' loop to consolidate account and raw number format. As some of these accounts do not contain data or dictionary, we need to handle the resulting TypeError and KeyError. Take care of indentation levels in a for loop

In [18]:
annual_is_stmts = []

# consolidate annual
for s in annual_is:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_is_stmts.append(statement)

Using a for loop only the data which is needed can be extracted and exported to a csv file

In [ ]:
annual_is_stmts[0]

Now Replicating this template for other Financial Statements as well by simply changing variable names and references

In [20]:
annual_cf_stmts = []
quarterly_cf_stmts = []

# annual cash flow statement
for s in annual_cf:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    annual_cf_stmts.append(statement)

# quarterly cash flow statement
for s in quarterly_cf:
    statement = {}
    for key, val in s.items():
        try:
            statement[key] = val['raw']
        except TypeError:
            continue
        except KeyError:
            continue
    quarterly_cf_stmts.append(statement)

In [ ]:
annual_cf_stmts[0]

# Extracting Profile Data

 Simply copying and pasting what we have already done and changing the url to profile url

In [22]:
response = requests.get(url_profile.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

Most of the interesting data is stored in QuoteSummaryStore. just like before

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore'].keys()

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile'].keys()

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['companyOfficers']

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['assetProfile']['longBusinessSummary']

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['secFilings']['filings']

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['summaryDetail']

# Statistics Data
Copying and Pasting the code for profile and making changes to the url

In [32]:
response = requests.get(url_stats.format(stock, stock))
soup = BeautifulSoup(response.text, 'html.parser')
pattern = re.compile(r'\s--\sData\s--\s')
script_data = soup.find('script', text=pattern).contents[0]
start = script_data.find("context")-2
json_data = json.loads(script_data[start:-12])

In [ ]:
json_data['context']['dispatcher']['stores']['QuoteSummaryStore']['defaultKeyStatistics']

# Extracting Historical Stock Data
Go to the historical data tab and copy the link of the download button on the top right hand side. 
This is an api call which can be seen from the 'query' prefix, the api version (v7 in this case) and parameters used in the query

In [38]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/KO?period1=1582157355&period2=1613779755&interval=1d&events=history'

Using a get request on stock url to get back csv formatted data, which can be saved in a csv file. 

In [39]:
response = requests.get(stock_url)

In [ ]:
response.text

This api can be customised to see different time intervals or granularity of time. 
To do that we need to split the query parameter from the url and put the query parameter into a python dictionary.

In [42]:
stock_url = 'https://query1.finance.yahoo.com/v7/finance/download/KO?'

params = {
    'period1':'1582157355',
    'period2':'1613779755',
    'interval':'1d',
    'events':'history'
}

These parameters can be customised further by removing the period parameters (which are simply time stamps), and replacing them with range parameters. In this case range is 5 years. We got a clue that we could use range from the chart tab as it had various selections available

In [43]:
params = {
    'range':'5y',
    'interval':'1d',
    'events':'history'
}

We are requesting now 5 years of daily stock data

In [44]:
response = requests.get(stock_url.format(stock), params=params)

In [ ]:
response.text

In [ ]:
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)
for row in data[:5]:
    print(row)

Trying different intervals now

In [50]:
params = {
    'range':'5y',
    'interval':'1wk',
    'events':'history'
}

In [51]:
response = requests.get(stock_url.format(stock), params=params)

In [ ]:
file = StringIO(response.text)
reader = csv.reader(file)
data = list(reader)
for row in data[:5]:
    print(row)

To find hidden apis, take a look at network activity on the webpage and investigate. Open up the developer tools (Right click -> Inspect) in the Summary page.
Go to the Network tab. If there are any apis calls being made, they will show up here, specifically under XHR. XHR is a javascript object that is used to transfer data between our browser and web server. 
Click on some items and a new screen displays Headers and Response. Look for a response that appears to be in a json formatted strings. 
Then go to the Headers to check what calls are being made, and also what parameters are being used in that api call. 
Make a mistake and experiment. It is possible that an api would give a feedback which could be used then.